In [9]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

In [10]:
from quantopian.pipeline import CustomFactor
import numpy

In [11]:
class StdDev(CustomFactor):
    def compute(self, today, asset_ids, out, values):
        # Calculates the column-wise standard deviation, ignoring NaNs
        out[:] = numpy.nanstd(values, axis=0)

In [12]:
def make_pipeline():
    std_dev = StdDev(inputs=[USEquityPricing.close], window_length=5)

    return Pipeline(
        columns={
            'std_dev': std_dev
        }
    )

In [13]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result.head()

std_dev
2015-05-05 00:00:00+00:00 Equity(2 [HWM])      0.293428
                          Equity(21 [AAME])    0.004714
                          Equity(24 [AAPL])    1.737677
                          Equity(25 [HWM_PR])  0.275000
                          Equity(31 [ABAX])    4.402971

In [14]:
class TenDayMeanDifference(CustomFactor):
    inputs = [USEquityPricing.close, USEquityPricing.open]
    window_length = 10
    def compute(self, today, asset_ids, out, close, open):
        out[:] = numpy.nanmean(close - open, axis=0)

In [15]:
close_open_diff = TenDayMeanDifference()

In [16]:
high_low_diff = TenDayMeanDifference(inputs=[USEquityPricing.high, USEquityPricing.low])

In [17]:
class Momentum(CustomFactor):
    inputs = [USEquityPricing.close]

    def compute(self, today, assets, out, close):
        out[:] = close[-1] / close[0]

In [18]:
ten_day_momentum = Momentum(window_length=10)
twenty_day_momentum = Momentum(window_length=20)

positive_momentum = ((ten_day_momentum > 1) & (twenty_day_momentum > 1))

In [19]:
def make_pipeline():

    ten_day_momentum = Momentum(window_length=10)
    twenty_day_momentum = Momentum(window_length=20)

    positive_momentum = ((ten_day_momentum > 1) & (twenty_day_momentum > 1))

    std_dev = StdDev(inputs=[USEquityPricing.close], window_length=5)

    return Pipeline(
        columns={
            'std_dev': std_dev,
            'ten_day_momentum': ten_day_momentum,
            'twenty_day_momentum': twenty_day_momentum
        },
        screen=positive_momentum
    )

In [21]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result.head()

std_dev  ten_day_momentum  \
2015-05-05 00:00:00+00:00 Equity(2 [HWM])    0.293428          1.036612   
                          Equity(24 [AAPL])  1.737677          1.014256   
                          Equity(39 [DDC])   0.138939          1.062261   
                          Equity(52 [ABM])   0.093680          1.009212   
                          Equity(64 [GOLD])  0.178034          1.025721   

                                             twenty_day_momentum  
2015-05-05 00:00:00+00:00 Equity(2 [HWM])               1.042783  
                          Equity(24 [AAPL])             1.021380  
                          Equity(39 [DDC])              1.167319  
                          Equity(52 [ABM])              1.015075  
                          Equity(64 [GOLD])             1.065587